In [54]:
# Usefull imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from hmmlearn import hmm
from IOHMM import IOHMM_model

In [55]:
hmm_model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=100)

hmm_model.startprob_ = np.array([0.5, 0.5])
hmm_model.transmat_ = np.array([[0.8, 0.2],
                                [0.2, 0.8]])
hmm_model.means_ = np.array([[0.0], [10.0]])
hmm_model.covars_ = np.array([[1.0], [1.0]])


In [35]:
#sample data
X, Z = hmm_model.sample(100)
#dataframe
data = pd.DataFrame(X, columns=['X'])
data['Z'] = Z

data.head(1000)

,X,Z
0,-0.301161,0
1,-0.934514,0
2,8.887644,1
3,9.957723,1
4,8.846005,1
...,...,...
95,0.587831,0
96,10.277797,1
97,10.353901,1
98,8.504641,1


In [51]:
# Torch  data
output = torch.tensor(data['X'].values, dtype=torch.float32)
# Input data ones
input = torch.ones(output.size(), dtype=torch.float32).unsqueeze(1)

input.shape, output.shape


(torch.Size([100, 1]), torch.Size([100]))

In [52]:
IOHMM = IOHMM_model(num_states=2, inputs=input, outputs=output, max_iter=100, tol=1e-4)


In [53]:
IOHMM._baum_welch()

Iteration 1, likelihood: -inf


/home/alessandro/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/alessandro/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/alessandro/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/alessandro/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/alessandro/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, 

RuntimeError: Function 'MulBackward0' returned nan values in its 1th output.